In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import constants as ct

# Limpeza dos dados

Dataset Original

In [2]:
df_gripe = pd.read_csv(ct.PATH_GRIPE, low_memory=False)

Dataset tratado

In [3]:
df_gripe_clean = df_gripe.copy()

Colunas constantes

In [4]:
constant_cols = [x  for x in df_gripe.columns if (len(df_gripe[x].unique())==1)]
constant_cols

['Número da notificação', 'IF - Positivo para outro vírus respiratório']

In [5]:
for c in constant_cols:
    if c in df_gripe_clean.columns:
        del df_gripe_clean[c] 

removed columns

In [6]:
print(len(constant_cols))

2


## Unnamed columns

Removemos as colunas sem nome

In [7]:
unnamed_cols = [x for x in df_gripe.columns if 'unnamed' in x.lower()]

In [8]:
for col in unnamed_cols:
    if col in df_gripe_clean.columns:
        del df_gripe_clean[col]

In [9]:
print(len(unnamed_cols))

16


## 'Data da notificação'

In [12]:
from datetime import datetime

In [13]:
COL_DATA_NOTIF = 'Data da notificação'

Há algumas inconsistências

Assumimos que a coluna é um timestamp e que alguns estão com um 0 a menos

Para as linhas em que data tem menos de 15 0s adicionamos 1, para que os datetimes estejam no mesmo horário

In [15]:
to_fix = np.log10(df_gripe[COL_DATA_NOTIF]) < 15
df_gripe_clean.loc[to_fix,COL_DATA_NOTIF] = df_gripe.loc[to_fix,COL_DATA_NOTIF]*10

In [73]:
np.sum(to_fix)

1843

Convertemos a data em datetime

In [16]:
df_gripe_clean[COL_DATA_NOTIF] = df_gripe_clean[COL_DATA_NOTIF].apply(lambda x:
                            datetime.fromtimestamp(x/1e6))

Range de Datas de Notificação

In [18]:
df_gripe_clean[COL_DATA_NOTIF].describe()

count                   17545
unique                     92
top       2020-05-07 22:06:40
freq                     1163
first     2020-01-02 18:20:00
last      2020-05-14 20:46:40
Name: Data da notificação, dtype: object

## 'Semana epidemiológica da notificação'

Por convenção internacional as semanas epidemiológicas são contadas de domingo a sábado.

In [19]:
COL_DATA_SEM_EP = "Semana epidemiológica da notificação"

Convertemos a data em datetime

In [58]:
df_gripe_clean[COL_DATA_SEM_EP] = pd.to_datetime(
    df_gripe[COL_DATA_SEM_EP], format='%d/%m/%Y')

Descrição das datas

In [59]:
df_gripe_clean[COL_DATA_SEM_EP].describe()

count                   17545
unique                    120
top       2020-04-27 00:00:00
freq                      695
first     2020-01-01 00:00:00
last      2020-05-14 00:00:00
Name: Semana epidemiológica da notificação, dtype: object

In [63]:
future_sem = df_gripe_clean[COL_DATA_SEM_EP] > df_gripe_clean[COL_DATA_NOTIF]

Alguns dos casos a semana epidemiológica da notificação vem depois da data de notificação.

Mas a diferença é de um dia geralmente

Exceto por um caso em que esse valor é maior que 14 dias. O que não faz sentido

In [68]:
diff_notif_sem = df_gripe_clean[future_sem][COL_DATA_SEM_EP]- df_gripe_clean[future_sem][COL_DATA_NOTIF]
diff_notif_sem

134      0 days 01:53:20
3954     0 days 08:06:40
4849     0 days 01:53:20
5689     0 days 00:33:20
6318     0 days 01:53:20
6987     0 days 01:53:20
7981     0 days 00:20:00
8167     0 days 01:40:00
8465     0 days 00:20:00
8576     0 days 00:20:00
11324    0 days 08:06:40
11518    0 days 01:26:40
12293    0 days 01:53:20
12819    0 days 01:53:20
13152    0 days 01:26:40
13674    0 days 01:53:20
13888    0 days 04:20:00
14303    0 days 04:20:00
15450    0 days 04:06:40
16306    0 days 01:40:00
16628    0 days 01:40:00
16737    0 days 01:40:00
17037    0 days 00:33:20
17158    0 days 04:06:40
17181   14 days 06:33:20
dtype: timedelta64[ns]

In [69]:
diff_notif_sem.max()

Timedelta('14 days 06:33:20')

## 'Data de nascimento'

Acho que deram o nome errado a coluna

In [70]:
COL_NASCIMENTO_WRONG = 'Data de nascimento'
df_gripe_clean[COL_NASCIMENTO_WRONG].unique()

array(['1 - Masculino', '2 - Feminino', '9 - Ignorado'], dtype=object)

In [54]:
df_gripe_clean[COL_NASCIMENTO_WRONG].value_counts()

1 - Masculino    9642
2 - Feminino     7884
9 - Ignorado       19
Name: Data de nascimento, dtype: int64

Renomeamos a coluna

In [55]:
COL_SEXO = '_sexo'
df_gripe_clean[COL_SEXO] = df_gripe_clean[COL_NASCIMENTO_WRONG] 

## 'Sexo'

Difícil saber do que se trata realmente, parece ser um código

In [75]:
COL_SEXO_WRONG = 'Sexo'
df_gripe_clean[COL_SEXO_WRONG].value_counts()

3034984    501
2298120    415
2696851    387
3005992    341
5717256    329
          ... 
12548        1
2288230      1
6539858      1
27014        1
6003494      1
Name: Sexo, Length: 447, dtype: int64

In [76]:
if COL_SEXO_WRONG in df_gripe_clean:
    del df_gripe_clean[COL_SEXO_WRONG]

## 'Município notificante'

A vasta maioria dos casos são do rio de janeiro neste dataset

In [82]:
COL_MUN_NOTIF = 'Município notificante'
df_gripe[COL_MUN_NOTIF].value_counts()

RJ    17475
SP       30
MG       20
PR        3
AL        2
SC        2
BA        2
ES        2
MT        1
PI        1
MA        1
SE        1
PA        1
RS        1
DF        1
GO        1
CE        1
Name: Município notificante, dtype: int64

Filtramos apenas os casos do rio de janeiro e excluímos a coluna

In [87]:
df_gripe_clean = df_gripe_clean[df_gripe_clean[COL_MUN_NOTIF]=='RJ']

In [88]:
if COL_MUN_NOTIF in df_gripe_clean:
    del df_gripe_clean[COL_MUN_NOTIF]

## 'Unidade da idade'

Parece ser a verdadeira coluna para data de nascimento

In [92]:
COL_UNIDADE_IDADE = 'Unidade da idade'
COL_DATA_NASCIMENTO = 'Data de Nascimento'
df_gripe['Unidade da idade']

0        16/04/1953
1        14/11/1960
2        26/12/1936
3        22/07/1978
4        05/11/1945
            ...    
17540    12/12/1950
17541    27/01/1951
17542    04/09/1979
17543    13/06/1943
17544    01/01/1939
Name: Unidade da idade, Length: 17545, dtype: object

In [91]:
df_gripe['Todos os sintomas'].isna().sum()

13221

In [89]:
list(df_gripe.columns)

['Número da notificação',
 'Data da notificação',
 'Semana epidemiológica da notificação',
 'Data dos primeiros sintomas',
 'Semana epidemiológica dos primeiros sintomas',
 'UF notificante',
 'Município notificante',
 'Código do município notificante',
 'Unidade notificante',
 'CNES da unidade notificante',
 'Sexo',
 'Data de nascimento',
 'Unidade da idade',
 'Idade',
 'Gestante',
 'Raça/cor',
 'Etnia',
 'Escolaridade',
 'Ocupação',
 'CBO da ocupação',
 'UF de residência',
 'Município de residência',
 'Código do município de residência',
 'Bairro',
 'Zona',
 'País de residência',
 'Código do país de residência',
 'Histórico de viagem internacional',
 'País viagem internacional',
 'Código país viagem internacional',
 'Local viagem internacional',
 'Data de partida viagem internacional',
 'Data de retorno viagem internacional',
 'Caso de surto de SG que evoluiu para SRAG',
 'Caso nosocomial - adquirido no hospital',
 'Contato com aves ou suínos',
 'Sintomas - febre',
 'Sintomas - tosse'

'IF - Positivo para

In [54]:
df_gripe.columns[df_gripe.isna().sum(axis=0)/len(df_gripe) > 0.9]

Index(['Escolaridade', 'CBO da ocupação', 'UF de residência',
       'Código país viagem internacional', 'Local viagem internacional',
       'Data de partida viagem internacional',
       'Data de retorno viagem internacional',
       'Caso de surto de SG que evoluiu para SRAG',
       'Fatores de risco - tem outros', '< 6 Meses: Mãe recebeu vacina',
       'Data da vacinação da mãe', '< 6 Meses: Mãe amamenta',
       'Data da vacinação dose única', 'Data da vacinação 1ª dose',
       'Data da vacinação 2ª dose', 'Usou antiviral para gripe',
       'Antiviral - todos', 'Uso de suporte ventilatório',
       'IF - Positivo para outros vírus',
       'IF - Positivo para vírus sincicial respiratório',
       'IF - Positivo para parainfluenza 1',
       'IF - Positivo para parainfluenza 2',
       'IF - Positivo para parainfluenza 3', 'IF - Positivo para adenovírus',
       'IF - Outro vírus respiratórios', 'IF - Todos os resultados',
       'RT-PCR - Outro subtipo da influenza A',
       

In [24]:
df_gripe['Unidade notificante'].unique()

array([330455, 330580, 330330, 330452, 330170, 330020, 330550, 330070,
       330200, 330620, 330350, 330285, 330520, 330490, 330610, 330390,
       330240, 221100, 330510, 330050, 330010, 330022, 330030, 330340,
       330430, 330411, 330630, 330100, 330470, 330190, 330575, 330110,
       355030, 330420, 530010, 330093, 330400, 330187, 330475, 330415,
       330220, 330210, 330515, 330130, 330600, 330080, 330270, 330320,
       330500, 330290, 330250, 330360, 330045, 330040, 330395, 314390,
       316370, 330540, 330185, 330060, 311340, 330090, 330280, 310150,
       355410, 330023, 330180, 330205, 330380, 292740, 150270, 330015,
       350390, 230440, 520620, 291920, 330370, 320120, 270430, 310620,
       316250, 330460, 330440, 354990, 313240, 510340, 420820, 312510,
       316720, 313670, 211130, 431490, 330225, 354850, 351340, 330150,
       410840, 351880, 355220, 354980, 350950, 351380, 420200, 320530,
       410420, 330310, 280030])

In [30]:
df_gripe['Semana epidemiológica da notificação'] = pd.to_datetime(df_gripe['Semana epidemiológica da notificação'])

In [31]:
df_gripe['Semana epidemiológica da notificação'].min()

Timestamp('2020-01-01 00:00:00')

In [32]:
df_gripe['Semana epidemiológica da notificação'].max()

Timestamp('2020-12-05 00:00:00')

In [5]:
df_gripe.sample(3)

,Número da notificação,Data da notificação,Semana epidemiológica da notificação,Data dos primeiros sintomas,Semana epidemiológica dos primeiros sintomas,UF notificante,Município notificante,Código do município notificante,Unidade notificante,CNES da unidade notificante,...,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161
13817,3,1586900000000000,12/04/2020,16,07/04/2020,15,RJ,CABO FRIO,330070,HOSPITAL SANTA IZABEL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9907,3,1587800000000000,14/04/2020,16,11/04/2020,15,RJ,SAO GONCALO,330490,HOSPITAL SAO GONCALO LTDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14816,3,1588900000000000,28/04/2020,18,27/04/2020,18,RJ,RIO DE JANEIRO,330455,UNIMED RIO PA BARRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_gripe.isna().sum(axis=0)

Número da notificação                               0
Data da notificação                                 0
Semana epidemiológica da notificação                0
Data dos primeiros sintomas                         0
Semana epidemiológica dos primeiros sintomas        0
                                                ...  
Unnamed: 157                                    17542
Unnamed: 158                                    17544
Unnamed: 159                                    17544
Unnamed: 160                                    17544
Unnamed: 161                                    17544
Length: 162, dtype: int64